## Scraping MLS Player Stats

Scraping www.mlssoccer.com/ for player statistics

### Import Necessary Modules

In [33]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from datetime import datetime 
from dateutil import parser
import os
import time
import itertools

### Scrape Statistics

#### Define Scraper and Helpers

In [2]:
def send_request(url, attempts=3):

    cnt = 0
    while cnt <= attempts:
        cnt += 1        
        # Send Request
        response = requests.get(url)
        # Check Status
        print(response.url, '\n', response.status_code, response.reason)
        if (response.status_code >= 200) and (response.status_code < 300):           
            return response
        # If bad status, pause before trying again
        print('Pause, then retry')
        time.pause(15)
    return None

In [80]:
#
def generate_param_combos(param_groups):
    params = []
    for combo in itertools.product(*param_groups):
        params.append({'page': combo[0],
                       'year': combo[1], 
                       'franchise': combo[2], 
                       'season_type': combo[3],
                       'group': combo[4]})
    return params

In [77]:
# 
def check_for_data(soup):
    even_rows = soup.findAll('tr', {'class': 'even'})
    if len(even_rows) == 0:
        return False
    return True    

#### Send Request to Base URL and Verify Site is Up

In [5]:
base_url = 'http://www.mlssoccer.com/stats/season'
response = requests.get(base_url)

In [6]:
response.status_code, response.url

(200, 'https://www.mlssoccer.com/stats/season')

#### Define parameter options

In [38]:
page = np.arange(0, 2)
year = np.arange(2007, 2018)
season_type = ['REG', 'PS']
group = ['goals', 'assists', 'shots', 'fouls', 'goalkeeping']

# Find and parse franchise popup menu to get list of teams
soup = BeautifulSoup(response.text, 'lxml')
franchise_select = soup.find('select', {'id': 'edit-franchise', 'name': 'franchise'})
franchise = []
for team in franchise_select.findAll('option')[1:]:
    franchise.append(team['value'])

In [72]:
odd_rows = soup.findAll('tr', {'class': 'odd'})
even_rows = soup.findAll('tr', {'class': 'even'})